In [1]:
import numpy as np
import pandas as pd
import re
from os.path import join

from tools import *

In [2]:
## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"

## Read Data

In [5]:
diag_df = read_data(join(read_prefix,"DIAGNOSES_ICD"))
diag_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [6]:
pres_df = read_data(join(read_prefix,"PRESCRIPTIONS"),dtype={'NDC':str})
pres_df.head()

/home/liu/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,00469061711,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,00056017275,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,00338055002,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,00054829725,20mg Tablet,20,mg,1,TAB,PO


In [7]:
diag_disease_all = diag_df[['ICD9_CODE']].dropna().drop_duplicates()
pres_drug_all = pres_df[['NDC']].dropna().drop_duplicates()
sample_disease_1000 = diag_disease_all.sample(n=1000, random_state=2019)
sample_drug_1000 = pres_drug_all.sample(n=1000, random_state=2019)

In [8]:
diag_df_1000 = diag_df[diag_df['ICD9_CODE'].isin(sample_disease_1000['ICD9_CODE'])]
pres_df_1000 = pres_df[pres_df['NDC'].isin(sample_drug_1000['NDC'])]

write2file(diag_df_1000,join(write_prefix,"diag_df_1kdisease"))
write2file(pres_df_1000,join(write_prefix,"pres_df_1kdrug"))

In [9]:
## select common 1000 users after filter 1000 drugs and diseases
common_user_all = inner_join(diag_df_1000[['SUBJECT_ID']].drop_duplicates(),\
                             pres_df_1000[['SUBJECT_ID']].drop_duplicates(),'SUBJECT_ID')
sample_user_1000 = common_user_all.sample(n=1000, random_state=2019)
diag_df_1k_1k = diag_df_1000[diag_df_1000['SUBJECT_ID'].isin(sample_user_1000['SUBJECT_ID'])]
pres_df_1k_1k = pres_df_1000[pres_df_1000['SUBJECT_ID'].isin(sample_user_1000['SUBJECT_ID'])]

In [10]:
write2file(diag_df_1k_1k,join(write_prefix,"diag_df_1k_1k"))
write2file(pres_df_1k_1k,join(write_prefix,"pres_df_1k_1k"))

## Diag_df_1000 -> patients*clusters  *  cluster*elements

In [3]:
diag_df_1k_1k = read_data(join(write_prefix,"diag_df_1k_1k"))
pres_df_1k_1k = read_data(join(write_prefix,"pres_df_1k_1k"))

In [4]:
diag_df_1k_1k.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1503,115,114585,2.0,1977
1,1509,115,114585,8.0,5119
2,586,72,156857,5.0,2762
3,591,72,156857,10.0,V290
4,592,72,156857,11.0,7470


In [5]:
## randomly select one HADM_ID for each patient
diag_hadm_all = inner_join(diag_df_1k_1k[['SUBJECT_ID','HADM_ID']].drop_duplicates(),\
                           pres_df_1k_1k[['SUBJECT_ID','HADM_ID']].drop_duplicates(),'SUBJECT_ID')
diag_hadm_all = diag_hadm_all[diag_hadm_all['HADM_ID_x']==diag_hadm_all['HADM_ID_y']]
diag_hadm_all.head()

,SUBJECT_ID,HADM_ID_x,HADM_ID_y
0,115,114585,114585
1,72,156857,156857
3,17,161087,161087
4,26,197661,197661
5,62,116009,116009


In [6]:
size = 1        # sample size
fn = lambda obj: obj.loc[np.random.choice(obj.index, size),:]
sample_hadm_1k = diag_hadm_all.groupby('SUBJECT_ID', as_index=False).apply(fn)['HADM_ID_x']

In [7]:
diag_df_1k_1k_hadm = diag_df_1k_1k[diag_df_1k_1k['HADM_ID']\
                                   .isin(sample_hadm_1k)][['SUBJECT_ID','ICD9_CODE']].drop_duplicates()
diag_df_1k_1k_hadm['VALUE']=1

In [8]:
diag_matrix = diag_df_1k_1k_hadm.pivot(index='SUBJECT_ID', columns='ICD9_CODE', values='VALUE').fillna(0).reset_index()
print(diag_matrix.shape)
diag_matrix.head()

(997, 289)


ICD9_CODE,SUBJECT_ID,0270,0392,04100,04109,04110,05319,07044,07070,07071,...,V290,V443,V4450,V4502,V5419,V550,V632,V643,V8522,V8543
0,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
pres_df_1k_1k_hadm = pres_df_1k_1k[pres_df_1k_1k['HADM_ID']\
                                   .isin(sample_hadm_1k)][['SUBJECT_ID','NDC']].drop_duplicates()
pres_df_1k_1k_hadm['VALUE']=1


In [10]:
pres_matrix = pres_df_1k_1k_hadm.pivot(index='SUBJECT_ID', columns='NDC', values='VALUE').fillna(0).reset_index()
print(pres_matrix.shape)
pres_matrix.head()


(997, 499)


NDC,SUBJECT_ID,2324030,2411533,2411733,2445485,2755901,2756101,3042950,3043730,3045051,...,67286417701,67919001101,68084004411,68084006501,68084007965,68084027801,68084030401,68516521102,74300000300,87701083336
0,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
write2file(diag_matrix,join(write_prefix,"diag_matrix"))

In [62]:
write2file(pres_matrix,join(write_prefix,"pres_matrix"))

## Running LDA 

In [3]:
diag_matrix = read_data(join(write_prefix, "diag_matrix"))
pres_matrix = read_data(join(write_prefix,"pres_matrix"))
diag_matrix.head()

,SUBJECT_ID,0270,0392,04100,04109,04110,05319,07044,07070,07071,...,V290,V443,V4450,V4502,V5419,V550,V632,V643,V8522,V8543
0,17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,72,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
from sklearn.decomposition import LatentDirichletAllocation

In [5]:
from numpy.linalg import norm

def cosine_sim(v1, v2):
    return np.matmul(v1,v2)/(norm(v1)*norm(v2))


In [21]:
def test_lda(diag_params, pres_params, n_comp =2):
    
#     n_comp = 2
    ## doc_topic_prior, control the sparsity of probability
    diag_topic, diag_word = diag_params
    lda = LatentDirichletAllocation(n_components=n_comp, random_state=2019,\
                                    doc_topic_prior=diag_topic,topic_word_prior=diag_word)
    diag_Z = lda.fit_transform(diag_matrix.iloc[:,1:]) 
    # print(diag_Z.shape)
#     print("The first 5 rows of diagnoses_Z:\n %s"%diag_Z[:5])

    pres_topic, pres_word = pres_params
    lda_2 = LatentDirichletAllocation(n_components=n_comp, random_state=2019,\
                                      doc_topic_prior=pres_topic,topic_word_prior=pres_word)
    pres_Z = lda_2.fit_transform(pres_matrix.iloc[:,1:]) 
    # print(pres_Z.shape)
#     print("The first 5 rows of prescriptions_Z:\n %s"%pres_Z[:5])

    for i in range(n_comp):
        l = []
        for j in range(n_comp):
            sim_i_j = cosine_sim(diag_Z[:,i],pres_Z[:,j])
    #         print(sim_i_j)
            l = l + [sim_i_j]
        current_range = np.max(l)-np.min(l)
        print("Similarity between diag_%d and pres_j(j∈[0,%d): %s"%(i,n_comp, l))
        print("Max:%f  Range:%f arg_max:%d"%(np.max(l),current_range,np.argmax(l)))
        
    return diag_Z, pres_Z

In [20]:
test_lda([0.1,0.001],[0.1,0.001],2)

Similarity between diag_0 and pres_j(j∈[0,2): [0.5094133093584077, 0.7021356526677573]
Max:0.702136  Range:0.192722 arg_max:1
Similarity between diag_1 and pres_j(j∈[0,2): [0.5484350005711016, 0.5268064023664191]
Max:0.548435  Range:0.021629 arg_max:0


In [28]:
for n_comp in range(2,11):
    print("=============================================================\nn_comp: %d"%n_comp)
    test_lda([0.1,0.001],[0.1,0.001],n_comp)    

 n_comp: 2
Similarity between diag_0 and pres_j(j∈[0,2): [0.5094133093584077, 0.7021356526677573]
Max:0.702136  Range:0.192722 arg_max:1
Similarity between diag_1 and pres_j(j∈[0,2): [0.5484350005711016, 0.5268064023664191]
Max:0.548435  Range:0.021629 arg_max:0
 n_comp: 3
Similarity between diag_0 and pres_j(j∈[0,3): [0.34639524166113755, 0.5344992463628864, 0.4574660286196103]
Max:0.534499  Range:0.188104 arg_max:1
Similarity between diag_1 and pres_j(j∈[0,3): [0.48733829929611666, 0.3969271033431372, 0.3900187033128421]
Max:0.487338  Range:0.097320 arg_max:0
Similarity between diag_2 and pres_j(j∈[0,3): [0.24779718944660944, 0.520215583601441, 0.4298543632375101]
Max:0.520216  Range:0.272418 arg_max:1
 n_comp: 4
Similarity between diag_0 and pres_j(j∈[0,4): [0.3438521175338138, 0.43375962013971103, 0.34437582325090355, 0.35735000089995367]
Max:0.433760  Range:0.089908 arg_max:1
Similarity between diag_1 and pres_j(j∈[0,4): [0.3547479090652208, 0.3501375876335105, 0.3375033978196364,

Similarity between diag_0 and pres_j(j∈[0,10): [0.1789592370384502, 0.228446719792865, 0.20250853731842972, 0.2032647727042648, 0.23793451666499812, 0.1590603795882128, 0.24230588136176734, 0.1985058209977379, 0.34834767027234875, 0.2204908083282439]
Max:0.348348  Range:0.189287 arg_max:8
Similarity between diag_1 and pres_j(j∈[0,10): [0.22180710983235286, 0.22015282391913024, 0.2462798842040567, 0.2095194082160021, 0.2726151942491745, 0.27234934225613533, 0.19500472633235416, 0.2012580238011624, 0.28554326234027594, 0.17780270323392894]
Max:0.285543  Range:0.107741 arg_max:8
Similarity between diag_2 and pres_j(j∈[0,10): [0.19742766662260666, 0.25687321950783937, 0.2502765090253432, 0.240194033855316, 0.22661013994220977, 0.23435663430024367, 0.20242648089262064, 0.20252733657089553, 0.2966963889784215, 0.20412375563593502]
Max:0.296696  Range:0.099269 arg_max:8
Similarity between diag_3 and pres_j(j∈[0,10): [0.1504646467343491, 0.26033606308659024, 0.2437564082418004, 0.1926592375618

In [29]:
## fix params for diagnoses, tune params for prescriptions
## increase by 0.1
for param in np.arange(0.2,1,0.2):
    print("pres_params: %s"%([param,param*0.01]))
    test_lda([0.1,0.001],[param,param*0.01],6)
## decrease by 0.01
for param in np.arange(0.01,0.1,0.02):
    print("pres_params: %s"%([param,param*0.01]))
    test_lda([0.1,0.001],[param,param*0.01],6) 

pres_params: [0.2, 0.002]
Similarity between diag_0 and pres_j(j∈[0,6): [0.23794268668982652, 0.37792265646661816, 0.32646504311750435, 0.25608745464886, 0.33103068159704524, 0.4040019205951752]
Max:0.404002  Range:0.166059 arg_max:5
Similarity between diag_1 and pres_j(j∈[0,6): [0.22972546680225062, 0.3444728826024219, 0.29889101224807096, 0.3343208999426793, 0.2561477339071451, 0.3445346423389311]
Max:0.344535  Range:0.114809 arg_max:5
Similarity between diag_2 and pres_j(j∈[0,6): [0.2076765006007531, 0.3637223954153636, 0.33499796245535185, 0.20579684032036377, 0.2480206712922126, 0.3828758305270146]
Max:0.382876  Range:0.177079 arg_max:5
Similarity between diag_3 and pres_j(j∈[0,6): [0.22232178453633933, 0.37083100966828875, 0.3278502878526503, 0.22522618078545575, 0.30212616527947, 0.3534501951994967]
Max:0.370831  Range:0.148509 arg_max:1
Similarity between diag_4 and pres_j(j∈[0,6): [0.3126793968469438, 0.3440961716892855, 0.31862669852729225, 0.27706283991936564, 0.295753808631

Similarity between diag_0 and pres_j(j∈[0,6): [0.21296765205164225, 0.350048246411303, 0.29630568493753034, 0.22018190625914805, 0.24656025582092403, 0.3900888749287322]
Max:0.390089  Range:0.177121 arg_max:5
Similarity between diag_1 and pres_j(j∈[0,6): [0.27463702508959603, 0.33125861635609877, 0.24018071442286246, 0.22180820997182932, 0.20511290660254147, 0.3193886375239077]
Max:0.331259  Range:0.126146 arg_max:1
Similarity between diag_2 and pres_j(j∈[0,6): [0.188045025597188, 0.33367845957389375, 0.28671419116748076, 0.17643123177092362, 0.1854340077147023, 0.368760857491464]
Max:0.368761  Range:0.192330 arg_max:5
Similarity between diag_3 and pres_j(j∈[0,6): [0.20692152050769472, 0.36080484619363895, 0.29931662161032385, 0.1623735860213571, 0.23449380287392316, 0.3184108544847087]
Max:0.360805  Range:0.198431 arg_max:1
Similarity between diag_4 and pres_j(j∈[0,6): [0.31735738071805625, 0.33346048589994653, 0.2844604220513264, 0.16788900157533312, 0.22339765859677094, 0.2582209431

In [8]:
diag_Y = lda.components_
diag_Y

array([[1.00000000e-03, 1.00100000e+00, 1.00000000e-03, 2.00100000e+00,
        1.00000000e-03, 1.00100000e+00, 3.00100000e+00, 1.16599914e+01,
        3.00100000e+00, 1.00000000e-03, 4.90040071e+00, 1.00000000e-03,
        1.00000000e-03, 1.00100000e+00, 1.00100000e+00, 1.00100000e+00,
        1.00100000e+00, 1.00100000e+00, 1.00000000e-03, 1.00000000e-03,
        2.19265705e+00, 1.00000000e-03, 1.00101709e+00, 1.00100000e+00,
        4.00096666e+00, 1.00000000e-03, 1.00100000e+00, 1.00000000e-03,
        4.00100000e+00, 1.00000000e-03, 1.00100000e+00, 1.00000000e-03,
        1.00000000e-03, 4.00101074e+00, 1.00096546e+00, 1.00100000e+00,
        1.00000000e-03, 1.00000000e-03, 1.00000000e-03, 1.00000000e-03,
        4.56967796e+00, 4.00100000e+00, 7.36594560e+01, 1.90009936e+01,
        3.00100000e+00, 2.00100000e+00, 1.00100000e+00, 1.00100000e+00,
        1.00000000e-03, 1.25001000e+02, 1.00000000e-03, 1.00000000e-03,
        1.00098386e+00, 1.00000000e-03, 5.72827220e+01, 1.000000